In [1]:
import re
import nltk
import gensim
import os
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
from pprint import pprint
nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')
import spacy
nlp = spacy.load('C:\Anaconda3\Lib\site-packages\en_core_web_sm/en_core_web_sm-3.0.0')
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

C:\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tlab\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)


def remove_stopwords(texts):

    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words if len(word) > 3] for doc in texts]


def lemmatization(texts, allowed_postags=None):
    if allowed_postags is None:
        allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])

    return texts_out

def make_bigrams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)  # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)  # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return [trigram_mod[bigram_mod[doc]] for doc in texts]



C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    coherence_values_cv = []
    coherence_values_umass = []

    
    model_list = []
    model_list_umass = []

    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                num_topics=num_topics,
                                                random_state=100,
                                                chunksize=200,
                                                passes=10,
                                                per_word_topics=True,
                                                id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values_cv.append(coherencemodel.get_coherence())
        coherencemodel_umass = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='u_mass')
        coherence_values_umass.append(coherencemodel_umass.get_coherence())



    return model_list, coherence_values_cv, model_list_umass, coherence_values_umass

def compute_coherence(corpus, dictionary, k, a, b):

    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k,
                                           random_state=100,
                                           chunksize=200,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)

    coherence_model = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')

    return coherence_model.get_coherence()


def format_topics_sentences(lda_model, filenames, corpus, texts):
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(lda_model[corpus]):
        row = row_list[0] if lda_model.per_word_topics else row_list
        row = sorted(row, key=lambda x: (x[1]), reverse=True)

        # for topic_num, prop_topic in row:
        #     wp = lda_model.show_topic(topic_num, topn=50)
        #     topic_keywords = ", ".join([word for word, prop in wp])
        #     sent_topics_df = sent_topics_df.append(
        #         pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords, filenames[i]]), ignore_index=True)

        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model.show_topic(topic_num, topn=50)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(
                    pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords, filenames[i]]), ignore_index=True)
            else:
                 break

    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'Papers']

    return sent_topics_df

C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df = pd.read_csv(r'C:\text_all_documents.csv')
filenames = df.filename.values.tolist()
data = df.description.values.tolist()
data_words = list(sent_to_words(data))
data_words_no_stops = remove_stopwords(data_words)
data_words_bigrams = make_trigrams(data_words_no_stops)
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])
id2word = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                    id2word=id2word,
                                                    num_topics=6,
                                                    alpha='asymmetric',
                                                    eta = 'symmetric',
                                                    random_state=100,
                                                    chunksize=200,
                                                    passes=10,
                                                    per_word_topics=True)
pprint(lda_model.print_topics())
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(coherence_lda)

C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.046*"werknemer" + 0.027*"artikel" + 0.020*"werkgever" + 0.010*"indien" + '
  '0.009*"recht" + 0.009*"grond" + 0.009*"jaar" + 0.007*"zoal" + 0.006*"loon" '
  '+ 0.005*"bedoeld_artikel"'),
 (1,
  '0.018*"instellingen" + 0.018*"benchmark" + 0.010*"medewerker" + '
  '0.010*"figuur" + 0.009*"sector" + 0.009*"bron" + 0.009*"studenten" + '
  '0.009*"jaar" + 0.009*"school" + 0.007*"deelnemer"'),
 (2,
  '0.025*"totale" + 0.011*"ouder" + 0.010*"school" + 0.009*"deelnemer" + '
  '0.008*"gewogen" + 0.007*"opzichte" + 0.007*"deelnemers" + '
  '0.005*"controller" + 0.005*"datum" + 0.005*"opleidingen"'),
 (3,
  '0.011*"student" + 0.007*"school" + 0.007*"instellingen" + 0.007*"niveau" + '
  '0.005*"binnen" + 0.005*"bestuur" + 0.005*"medewerker" + 0.005*"werk" + '
  '0.004*"jaar" + 0.004*"code"'),
 (4,
  '0.036*"fusie" + 0.013*"fusietoet" + 0.012*"cfto" + 0.009*"besturen" + '
  '0.009*"leerlingen" + 0.007*"scholen" + 0.007*"bestuur" + 0.007*"minister" + '
  '0.006*"school" + 0.006*"advie"'),


In [8]:
# Visualize the topics
pyLDAvis.enable_notebook(local=True)
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared


C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.083285 -0.013927       1        1  27.386359
5     -0.221918 -0.038362       2        1  17.802005
4      0.062650 -0.033927       3        1  16.761902
2      0.053779  0.156162       4        1  16.459223
3      0.060189  0.101646       5        1  14.160158
0      0.128584 -0.171592       6        1   7.430353, topic_info=           Term         Freq        Total Category  logprob  loglift
183     diploma  2646.000000  2646.000000  Default  30.0000  30.0000
4858      fusie  1604.000000  1604.000000  Default  29.0000  29.0000
480      niveau  2756.000000  2756.000000  Default  28.0000  28.0000
1787  werknemer   949.000000   949.000000  Default  27.0000  27.0000
728        vmbo  1153.000000  1153.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
97      bijlage    65.509062   481.429802   Topic6  -5.6871   0.6050
299       geval    54.422450   314.341726   Topic6  -5.8725   0.8459
731    volgende    54.522452   370.435910   Topic6  -5.8707   0.6835
442       maakt    52.730948   274.237850   Topic6  -5.9041   0.9508
53        basis    52.900962   689.527752   Topic6  -5.9009   0.0320

[442 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
3         1  0.583801       aandeel
3         2  0.358283       aandeel
3         3  0.038193       aandeel
3         4  0.005456       aandeel
3         5  0.003637       aandeel
...     ...       ...           ...
2007      3  0.141653          zorg
2007      4  0.146222          zorg
2007      5  0.571181          zorg
2007      6  0.045695          zorg
6076      6  0.914488  zwangerschap

[1136 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 5, 3, 4, 1])

In [9]:
df_topic_sents_keywords = format_topics_sentences(lda_model=lda_model, filenames=filenames, corpus=corpus, texts=data_lemmatized)
#
# # Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Papers']

# Show
df_dominant_topic.head(10)

df_dominant_topic_distribution = df_dominant_topic['Dominant_Topic'].value_counts().reset_index(name="Num Documents")
df_dominant_topic_distribution.columns = ["Dominant_Topic", "Num Documents"]
df_dominant_topic_distribution.sort_values("Dominant_Topic")
print(df_dominant_topic_distribution)

topics = df_dominant_topic.groupby(['Dominant_Topic'])
# # print(len(topics))
with open('keywords_per_topic_6.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["KEYWORDS", "PAPERS"])

    for topic_key, topic_value in topics:
        group = topics.get_group(topic_key)

        writer.writerow([group.iloc[0]['Keywords'], group['Papers'].tolist()])


C:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   Dominant_Topic  Num Documents
0             1.0             33
1             3.0             25
2             2.0             19
3             5.0             19
4             0.0             13
5             4.0             12


In [ ]:
# FINETUNING
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 6
max_topics = 7
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
r1, r2 = 0.01, 1.01
# Alpha parameter
alpha = list(np.arange(r1, r2, 0.1))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(r1, r2, 0.1))

beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [  # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25),
        # gensim.utils.ClippedCorpus(bow_corpus, int(num_of_docs*0.5)),
        # gensim.utils.ClippedCorpus(corpus, num_of_docs * 0.75),
        corpus]
corpus_title = ['100% Corpus']
model_results = {'Validation_Set': [],
                     'Topics': [],
                     'Alpha': [],
                     'Beta': [],
                     'Coherence': []
                     }
if 1 == 1:
    pbar = tqdm(total=5000)    
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence(corpus=corpus, dictionary=id2word, k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results_umass.csv', index=False)
    pbar.close()